In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
SOS_token = 0
EOS_token = 1

In [4]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0 : "SOS", 1 : "EOS"}
        self.n_words = 2
        
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
            
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [5]:
def unicodeToAscii(s):
        return ''.join(
            c for c in unicodedata.normalize('NFD', s)
            if unicodedata.category(c) != 'Mn'
        )
    
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s
    
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")
    
    lines = open('./%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')
        
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
        
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
            
    return input_lang, output_lang, pairs

In [6]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and\
        len(p[1].split(' ')) < MAX_LENGTH and\
        p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [7]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['c est une belle fille .', 'she s a beauty .']


In [13]:
a = torch.tensor([[3], [1], [2], [4]])

print(a.shape)
print(a.size())
print(a.size(0))

torch.Size([4, 1])
torch.Size([4, 1])
4


In [15]:
b1 = torch.tensor([[1], [2]])
b2 = torch.tensor([[3], [4], [5]])
b3 = torch.tensor([6])

list = [b1, b2, b3]
print(list)

[tensor([[1],
        [2]]), tensor([[3],
        [4],
        [5]]), tensor([6])]


In [22]:
v = torch.tensor([[1, 2, 3, 4]])
w = torch.tensor([[0.1, 0.2, 0.3, 0.4], [0.3, 0.4, 0.5, 0.6]]).long()
test = v.matmul(w) + 3
print(test)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x4 and 2x4)

In [39]:
v = torch.tensor([[0.1, 0.2, 0.3, 0.4]], dtype = torch.float)
w = torch.tensor([2], dtype = torch.long)
NLLLoss = nn.NLLLoss()

print(NLLLoss(v, w))

tensor(-0.3000)


In [3]:
def normalizeString(s):
    s = s.lower().strip()
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^0-9a-zA-Z.!?]+", r" ", s)
    return s



few,  haha
